In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv(r'raw02.csv', index_col=False)
df.head()

,addr_pct_cd,cmplnt_fr_dt,cmplnt_fr_tm,crm_atpt_cptd_cd,juris_desc,ky_cd,law_cat_cd,loc_of_occur_desc,ofns_desc,rpt_dt,...,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,boro_nm,patrol_boro,pd_cd,pd_desc,prem_typ_desc,cmplnt_to_dt,cmplnt_to_tm
0,69,2021-12-17T00:00:00.000,22:13:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-17T00:00:00.000,...,2.0,25.0,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113,2021-12-17T00:00:00.000,06:21:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-17T00:00:00.000,...,3.0,46.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,113,2021-12-13T00:00:00.000,20:05:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-13T00:00:00.000,...,3.0,46.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,2021-12-07T00:00:00.000,22:49:00,COMPLETED,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,2021-12-07T00:00:00.000,...,5.0,40.0,33.0,BRONX,PATROL BORO BRONX,792.0,WEAPONS POSSESSION 1 & 2,STREET,NaN,NaN
4,113,2021-12-06T00:00:00.000,17:25:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-06T00:00:00.000,...,3.0,6.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['cmplnt_fr_dt','cmplnt_fr_tm','law_cat_cd','ofns_desc','latitude','longitude','boro_nm']]
df.head()

,cmplnt_fr_dt,cmplnt_fr_tm,law_cat_cd,ofns_desc,latitude,longitude,boro_nm
0,2021-12-17T00:00:00.000,22:13:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.645647,-73.902876,NaN
1,2021-12-17T00:00:00.000,06:21:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.682499,-73.787269,NaN
2,2021-12-13T00:00:00.000,20:05:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.689218,-73.791449,NaN
3,2021-12-07T00:00:00.000,22:49:00,FELONY,DANGEROUS WEAPONS,40.882718,-73.881247,BRONX
4,2021-12-06T00:00:00.000,17:25:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.696483,-73.776680,NaN


In [4]:
def convertClassToNum(p_df):
    p_df['datetime']=pd.to_datetime(p_df['cmplnt_fr_dt'] + ' ' + p_df['cmplnt_fr_tm'],errors='coerce')
    p_df=p_df.drop(columns=['cmplnt_fr_dt','cmplnt_fr_tm'])
    p_df=p_df.sort_values(by='datetime',ascending=True) 

    p_df['year']=pd.DatetimeIndex(p_df['datetime']).year
    p_df['month'] = pd.DatetimeIndex(p_df['datetime']).month
    p_df['weekday'] = p_df['datetime'].dt.dayofweek
    p_df['Hour'] = p_df.datetime.dt.hour
    p_df['Minute'] = p_df.datetime.dt.minute
    p_df=p_df.drop(columns=['datetime'])

    list_law_cat_cd=list(p_df['law_cat_cd'].unique())
    global dict_law_cat_cd
    dict_law_cat_cd = {}
    for i in list_law_cat_cd:
        dict_law_cat_cd[i] = list_law_cat_cd.index(i)
    p_df['law_cat_cd']=p_df['law_cat_cd'].map(dict_law_cat_cd)
#     print(dict_law_cat_cd)

    list_ofns_desc=list(p_df['ofns_desc'].unique())
    global dict_ofns_desc
    dict_ofns_desc = {}
    for i in list_ofns_desc:
        dict_ofns_desc[i] = list_ofns_desc.index(i)
    p_df['ofns_desc']=p_df['ofns_desc'].map(dict_ofns_desc)
#     print(dict_ofns_desc)

    p_df=p_df[p_df['year'].notna()]
    p_df=p_df.fillna("OTHERS")
    global dict_boro_nm
    dict_boro_nm = {'OTHERS':0,'BROOKLYN':1,'QUEENS':2,'BRONX':3,'MANHATTAN':4,'STATEN ISLAND':5}
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm)
#     print(dict_boro_nm)

    return p_df

In [5]:
df = convertClassToNum(df) 

print(df.describe())

######## Rearrange Columns #######
cols = df.columns.tolist()
cols = cols[2:10] + cols[0:2]
df = df[cols]

df.head()

          law_cat_cd      ofns_desc       latitude      longitude  \
count  449498.000000  449498.000000  449498.000000  449498.000000   
mean        0.836549       7.503924      40.736834     -73.923447   
std         0.686656       7.678109       0.082496       0.076968   
min         0.000000       0.000000      40.499401     -74.253767   
25%         0.000000       2.000000      40.675213     -73.971681   
50%         1.000000       4.000000      40.735019     -73.926464   
75%         1.000000      10.000000      40.811485     -73.879894   
max         2.000000      62.000000      40.912723     -73.700568   

             boro_nm           year          month        weekday  \
count  449498.000000  449498.000000  449498.000000  449498.000000   
mean        2.541544    2020.944950       6.823114       2.974759   
std         1.253370       0.880155       3.406422       1.975478   
min         0.000000    1920.000000       1.000000       0.000000   
25%         1.000000    2021.0000

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd,ofns_desc
83454,40.723009,-73.978088,4,1920.0,4.0,4.0,0.0,0.0,0,0
446163,40.748923,-73.892230,2,1921.0,1.0,6.0,8.0,50.0,1,1
446108,40.652309,-73.956290,1,1921.0,1.0,3.0,22.0,0.0,1,2
389152,40.840984,-73.925974,3,1921.0,2.0,0.0,23.0,47.0,1,3
375382,40.758800,-73.890222,2,1921.0,3.0,2.0,4.0,4.0,0,4


In [6]:
Train = df.head(359598)
Test = df.tail(89900)

In [7]:
print(Train.describe())
Train.head()

            latitude      longitude        boro_nm           year  \
count  359598.000000  359598.000000  359598.000000  359598.000000   
mean       40.736721     -73.923697       2.542361    2020.931187   
std         0.082787       0.076992       1.255501       0.983562   
min        40.499401     -74.253767       0.000000    1920.000000   
25%        40.674808     -73.971308       1.000000    2021.000000   
50%        40.734646     -73.926874       2.000000    2021.000000   
75%        40.811947     -73.880794       4.000000    2021.000000   
max        40.912723     -73.700568       5.000000    2021.000000   

               month        weekday           Hour         Minute  \
count  359598.000000  359598.000000  359598.000000  359598.000000   
mean        5.711275       2.973151      13.134094      20.232546   
std         2.864219       1.973699       6.478400      18.671251   
min         1.000000       0.000000       0.000000       0.000000   
25%         3.000000       1.0000

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd,ofns_desc
83454,40.723009,-73.978088,4,1920.0,4.0,4.0,0.0,0.0,0,0
446163,40.748923,-73.892230,2,1921.0,1.0,6.0,8.0,50.0,1,1
446108,40.652309,-73.956290,1,1921.0,1.0,3.0,22.0,0.0,1,2
389152,40.840984,-73.925974,3,1921.0,2.0,0.0,23.0,47.0,1,3
375382,40.758800,-73.890222,2,1921.0,3.0,2.0,4.0,4.0,0,4


In [8]:
print(Test.describe())
Test.head()

           latitude     longitude       boro_nm     year         month  \
count  89900.000000  89900.000000  89900.000000  89900.0  89900.000000   
mean      40.737285    -73.922446      2.538276   2021.0     11.270445   
std        0.081321      0.076864      1.244811      0.0      0.692789   
min       40.501101    -74.253257      0.000000   2021.0     10.000000   
25%       40.676401    -73.972868      1.000000   2021.0     11.000000   
50%       40.735902    -73.925111      2.000000   2021.0     11.000000   
75%       40.810298    -73.876845      4.000000   2021.0     12.000000   
max       40.912723    -73.700720      5.000000   2021.0     12.000000   

            weekday          Hour        Minute    law_cat_cd     ofns_desc  
count  89900.000000  89900.000000  89900.000000  89900.000000  89900.000000  
mean       2.981190     13.060011     20.631357      0.818242      7.359711  
std        1.982575      6.396563     18.595840      0.688815      7.592476  
min        0.000000  

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd,ofns_desc
91695,40.712308,-73.807608,2,2021.0,10.0,4.0,12.0,27.0,1,27
100040,40.712104,-73.806971,2,2021.0,10.0,4.0,12.0,27.0,1,27
93971,40.647020,-73.958124,1,2021.0,10.0,4.0,12.0,28.0,1,1
92293,40.766374,-73.930638,2,2021.0,10.0,4.0,12.0,30.0,2,7
100992,40.683131,-73.968871,1,2021.0,10.0,4.0,12.0,30.0,2,7


In [10]:
Train.to_csv('converted_traindata.csv')
Test.to_csv('converted_testdata.csv')